In [ ]:
import os 
import subprocess
from multiprocessing.pool import ThreadPool

path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher"
path_pdb = f"{path_work}/ficheros_28032023/output_ppt"
path_out = f"{path_work}/ficheros_28032023/sword2_DepoSeq"


def sword2_pred(pdb_file) :
    """
    This function makes a sword prediction from a pdb_file.
    It takes as an input a pdb file, and outputs swords2 files.     
    The path_out is to be adjusted to the context.
    """
    with open(f"{path_out}/sword2.log" ,"a+") as outfile :
        path_sword = "/home/conchae/software/SWORD2"
        name = pdb_file.split("/")[-1].split(".pdb")[0]
        sword_command = f" {path_sword}/SWORD2.py -i {pdb_file} -o {path_out}/{name} -x 5"
        sword_subprocess = subprocess.Popen (sword_command , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
        hmm_out, hmm_err = sword_subprocess.communicate()
        print(hmm_out)
        outfile.write(f"{name} is done.")
    
paths = [f"{path_pdb}/{file}" for file in os.listdir(path_pdb)]
paths = [f"{path_pdb}/{file}" for file in os.listdir(path_pdb) if os.path.isdir(f"{path_out}/{file.split('.pdb')[0]}")==False]

    
if __name__ == '__main__':
    with ThreadPool(10) as p:
        p.map(sword2_pred, paths)
        

In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
path_pdb = f"{path_project}/sword2_DepoSeq_pt2"
path_db = "/home/conchae/depolymerase/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp/tmp_mini"

def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan_domains.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/seekfold_PPT_domains"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_domain = []
for file in os.listdir(f"{path_pdb}") :
    try : 
        for file_2 in os.listdir(f"{path_pdb}/{file}/{file}_A/Protein_Units") :
            if file_2[-3:] == "pdb" :
                file_2_name = file_2.split(".pdb")[0]
                if os.path.isfile(f"{path_project}/seekfold_PPT_domains/{file_2_name}.out") == False :
                    path = f"{path_pdb}/{file}/{file}_A/Protein_Units/{file_2}"
                    paths_domain.append(path)
    except Exception as e:
        print(file)

if __name__ == '__main__':
    with ThreadPool(30) as p:
        p.map(seek_beta_helix, paths_domain)

In [ ]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"

# ********************************************************
# PPT version : 
dir_out_full = f"{path_project}/seekfold_PPT_full"
dir_out = f"{path_project}/seekfold_PPT_domains"
path_swords_out = f"{path_project}/sword2_DepoSeq_pt2"
# *****************************************************
# Minibatch version :
dir_out_full = f"{path_project}/seekfold_minibatch_full"
dir_out = f"{path_project}/seekfold_minibatch_domains"
path_swords_out = f"{path_project}/sword2_Minibatch_pt2"
# *********************************************************

path_info = f"/home/conchae/depolymerase/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob","aln_score"]
dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

errors = {'A1l_00058_A_1_129_274.pdb',
 'AF-A0A089GJA0-F1-model_v4.pdb',
 'AF-A0A4V2M274-F1-model_v4.pdb',
 'AF-A0A7W9JCS6-F1-model_v4.pdb','AF-A0A658Y395-F1-model_v4.pdb'}

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")

dico_domain_result = {}
for protein in tqdm(dico_out) :
    the_winner = (0,0,0)
    len_protein = len(list(SeqIO.parse(f"{path_swords_out}/{protein}/{protein}_A/SWORD/{protein}_A/{protein}_A.fasta", "fasta"))[0])
    # Collecting data for each proteins
    results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}.out" , sep = "\t" , names = header_seekfold).fillna("wtf")
    results_full_prot = results_full_prot[~results_full_prot["target"].isin(errors)]
    for _,row in results_full_prot.iterrows():
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            #full_score = int(row["bits"])**1.2 / len_protein**0.40
            full_score = int(row["bits"])
            target = row["target"]
            break
    the_full_prot = (full_score , f"len alignemnt : {row['alnlen']}" ,f"Full protein {protein}")
    for path in dico_out[protein] :
        if os.path.getsize(path) > 0 :
            results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold).fillna("wtf")
            results_df = results_df[results_df["target"] == target]
            if results_df.empty == False :
                aa_end = results_df["query"].values[0].split("_")[-1].split(".pdb")[0]
                aa_start = results_df["query"].values[0].split("_")[-2]
                PU_length = int(aa_end) - int(aa_start)
                #score = int(results_df["bits"].values[0])**1.2 / PU_length**0.40
                score = int(results_df["bits"].values[0]) 
                contender = (score , PU_length ,f"len alignemnt : {results_df['alnlen'].values[0]}",results_df["query"].values[0])
                if contender[0] > the_winner[0] :
                    the_winner = contender
                elif contender[0] == the_winner[0] :
                    if contender[1] < the_winner[1] :
                        the_winner = contender
                else :
                    pass
    if the_full_prot[0] > (the_winner[0] + 25) :
    #if the_full_prot[0] > (the_winner[0] + 25) :
        the_winner = the_full_prot 
    if the_winner[0] > 0 :
        dico_domain_result[protein] = the_winner